In [1]:
# Business logic applied to ERSS file:
# We take a distinct count of a primary key based on an obfuscated SSN.
# 0. Deduplicate the records based on calStateEduPersonUID (there doesn't seem to be any duplicates based on ID, Year, and Term.)
# 1. erss_campus NOT IN (7, 97, 93): This includes all campuses including CalStateTEACH with an educator preparation program
# 2. erss_cred_stat IN ('4','5','6','8', 'V'): This identifies students in a teaching credential program
# 3. erss_year = 2021: Chosen to align with erss year 2021 in the Teaching Credentials Enrollment Dashboard
# 4. erss_term = 4: Chosen to align with the Fall term in the Teaching Credentials Enrollment Dashboard
# 5. erss_cred_obj mapped to SourceCode (MS, ES, and SS flag) using the "Credential and Subject Matter Waiver Objective" data dictionary definition

# For NEW students:
# 6. ["erss_enroll_stat"].isin([4,5])]

In [2]:
# to format using black
# black scripts\erss_black_student_success.ipynb

# Load in libraries

In [1]:
# import pandas and pandasql libraries
import pandas as pd
from pandasql import sqldf

# Part 1: Load in raw enrollment data for AY 2021-2022

In [5]:
# read in raw data
df = pd.read_csv(
    "..\data\erss\ERSS_20213_20222_221215.csv",
    dtype={
        "erss_cred_stat": str,
        "erss_ethnic_old": str,
        "erss_cred_emph": str,
        "erss_spec_prog": str,
    },
)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\2299733542.py:3: SyntaxWarning: invalid escape sequence '\d'
  "..\data\erss\ERSS_20213_20222_221215.csv",


## 1a: Load in credential objective lookup table
- Used to determine MS,ES,SS down the line

In [6]:
# load in credential objective lookup table
df_lookup = pd.read_excel(
    "..\data\credential_objective_lookup\erss_cred_obj_lookup.xlsx",
    sheet_name="appendix_c_ir",
)

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\1471605555.py:3: SyntaxWarning: invalid escape sequence '\d'
  "..\data\credential_objective_lookup\erss_cred_obj_lookup.xlsx",


In [9]:
# change datatype of erss_cred_obj to int so it can be joined on the erss table
df_lookup["erss_cred_obj"] = df_lookup["erss_cred_obj"].astype(int)

## 1b: Load in campus names and codes lookup table
- Used to attach campus names to records

In [8]:
# load in campus names and campus codes
df_campus_codes = pd.read_excel("..\data\campus_codes\campus_codes_and_names.xlsx")

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\2539122274.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_campus_codes = pd.read_excel("..\data\campus_codes\campus_codes_and_names.xlsx")


# Part 2: Separate out records into INTERN, INTEGRATED, UNDERGRAD in REGULAR PROGRAMS, and POSTBACCs + GRADUATEs in REGULAR PROGRAMS

## 2a: INTERN

In [10]:
# recreate intern only
## filter down to teacher credential enrollments
df_2021_fall_intern = df[
    (df["erss_year"] == 2021)
    & (df["erss_term"] == 4)
    & (df["erss_cred_stat"].isin(["8"]))
    & (df["erss_stud_lev"] == 5)
]

In [11]:
# left join the program type onto the main DataFrame
df_2021_combined_intern = pd.merge(
    df_2021_fall_intern,
    df_lookup,
    left_on="erss_cred_obj",
    right_on="erss_cred_obj",
    how="left",
)

In [12]:
df_2021_combined_intern["code_value"].value_counts()

code_value
ES       348
SS       209
MS       167
Other     20
Name: count, dtype: int64

## 2b: INTEGRATED

In [13]:
# recreate integrated only
## filter down to teacher credential enrollments
df_2021_fall_integ = df[
    (df["erss_year"] == 2021)
    & (df["erss_term"] == 4)
    & (df["erss_cred_stat"].isin(["5", "6"]))
    & (df["erss_stud_lev"].isin([3, 4]))
    & (df["erss_stud_stand"].isin(["B"]))
]

In [14]:
# left join the program type onto the main DataFrame
df_2021_combined_integ = pd.merge(
    df_2021_fall_integ,
    df_lookup,
    left_on="erss_cred_obj",
    right_on="erss_cred_obj",
    how="left",
)

In [15]:
df_2021_combined_integ["code_value"].value_counts()

code_value
MS    510
ES     53
SS     52
Name: count, dtype: int64

### This means that the ES mismatch is likely due to a student in the REGULAR pool being counted.

# 2c: UNDERGRAD in REGULAR PROGRAMS

In [16]:
## filter down to teacher credential enrollments
## Undergrad
## stud_lev 3 == junior
## stud_lev 4 == senior

df_2021_fall_undergrad = df[
    (df["erss_year"] == 2021)
    & (df["erss_term"] == 4)
    & (df["erss_stud_lev"].isin([3, 4]))
    & (df["erss_cred_stat"].isin(["5", "6"]))
    & (df["erss_stud_stand"].isin(["5"]))
]

## 2d: POSTBACC AND GRADUATES IN REGULAR PROGRAMS

In [17]:
## Postbacc and graduates
## filter down to teacher credential enrollments

df_2021_fall_postbacc = df[
    (df["erss_year"] == 2021)
    & (df["erss_term"] == 4)
    & (df["erss_stud_lev"].isin([5]))
    & (df["erss_cred_stat"].isin(["4","5", "6", "V", "H", "I", "J", "K"]))
    & (df["erss_stud_stand"].isin(["C", "5", "1", "2", "3", "6", "7", "8"]))
]

In [18]:
df_2021_fall_postbacc.erss_cred_stat.value_counts()

erss_cred_stat
5    11349
4      456
V      408
6       30
Name: count, dtype: int64

# Part 3: Combine 2a: Intern and 2d: Postbacc and Graduates in regular programs

In [79]:
# combine A) INTERN and D) postbacc/graduate table together
# Why? Because this will only include (df["erss_stud_lev"].isin([5])) aka those NOT in undergraduate programs.
df_2021_fall = sqldf(
    """
    SELECT *
    FROM df_2021_fall_intern
    UNION
    SELECT * FROM df_2021_fall_postbacc
"""
)

In [80]:
# left join the program type onto the main DataFrame
df_2021_combined = pd.merge(
    df_2021_fall,
    df_lookup,
    left_on="erss_cred_obj",
    right_on="erss_cred_obj",
    how="left",
)

In [81]:
df_2021_combined["code_value"].value_counts()

code_value
MS       4816
SS       4302
ES       2244
Other    1625
Name: count, dtype: int64

In [23]:
df_2021_combined["erss_stud_lev"].value_counts()

erss_stud_lev
5    12987
Name: count, dtype: int64

In [24]:
# left join the campus names onto the main DataFrame

df_2021_combined_names = pd.merge(
    df_2021_combined,
    df_campus_codes,
    left_on="erss_campus",
    right_on="campus_code",
    how="left",
)

In [25]:
# filter down to MS, SS, ES credential type

df_2021_combined_names_MS_SS_ES_only = df_2021_combined_names[
    df_2021_combined_names["code_value"].isin(["MS", "SS", "ES"])
]

In [26]:
len(df_2021_combined_names_MS_SS_ES_only)

11362

In [ ]:
# df_2021_combined_names_MS_SS_ES_only.campus_code.value_counts()

### Validate enrollment counts for Monterey Bay and East Bay
- counts seemed to be low compared to other campuses

In [28]:
# new_students, all cred_stat
df_2021_test = df_2021_combined_names_MS_SS_ES_only[
    df_2021_combined_names_MS_SS_ES_only["erss_enroll_stat"].isin([4, 5])
]

In [29]:
# race by campus
sql_test_df= sqldf(
    """
    SELECT 
        campus_name,
        erss_cred_stat,
        COUNT(calstateEduPersonUID) as total_records
        FROM df_2021_test
        WHERE campus_name IN ('Monterey Bay', 'East Bay')
        GROUP BY 
            erss_cred_stat,
            campus_name
        ORDER BY 
            erss_cred_stat,
            campus_name
            """
)

In [30]:
sql_test_df

,campus_name,erss_cred_stat,total_records
0,Monterey Bay,4,11
1,East Bay,5,14
2,Monterey Bay,5,17
3,Monterey Bay,8,2
4,East Bay,V,2


# Part 4: Filter down to new students only

In [31]:
# filter down to first-time enrollments
# first-time enrollment is defined as
# erss_enroll_stat == 5: A student who is classified as a postbaccalaureate student for the first time,
# who has all units attempted recorded prior to receiving the bachelor's degree. OPTION: A postbaccalaureate student with units attempted after
# receiving the bachelor's degree who enters a graduate program can be coded as a first-time graduate student.
# erss_enroll_stat == 4: A postbaccalaureate student new to this campus this term who has units attempted elsewhere as a postbaccalaureate student.

df_2021_new_only = df_2021_combined_names_MS_SS_ES_only[
    df_2021_combined_names_MS_SS_ES_only["erss_enroll_stat"].isin([4, 5])
    # remove those who have cred_stat = "4" because they already have a preliminary credential.
    & (df_2021_combined_names_MS_SS_ES_only["erss_cred_stat"].isin(["8","5", "6", "V", "H", "I", "J", "K"]))
]

In [32]:
df_2021_new_only.erss_cred_stat.value_counts()

erss_cred_stat
5    4789
8      82
V      23
6       7
Name: count, dtype: int64

In [35]:
# 2 is Black
df_2021_new_only.erss_ipeds_race_catg.value_counts()

erss_ipeds_race_catg
7    2350
1    1629
4     430
8     203
6     165
2     109
3       9
5       6
Name: count, dtype: int64

## 4a: Load in IPEDS Race Lookup table

In [36]:
# read in ipeds lookup table
ipeds_race_df = pd.read_excel("..\data\ipeds_race_lookup\ipeds_race_lookup.xlsx")

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\4264161771.py:2: SyntaxWarning: invalid escape sequence '\d'
  ipeds_race_df = pd.read_excel("..\data\ipeds_race_lookup\ipeds_race_lookup.xlsx")


In [37]:
# join race descriptions onto the DataFrame
df_2021_new_race = pd.merge(
    df_2021_new_only,
    ipeds_race_df,
    left_on="erss_ipeds_race_catg",
    right_on="ipeds_value",
    how="left",
)

In [38]:
df_2021_new_race.race_description.value_counts()

race_description
Hispanic/Latino                              2350
White                                        1629
Asian                                         430
Unknown                                       203
Two or More Races                             165
Black or African American                     109
American Indian or Alaska Native                9
Native Hawaiian or Other Pacific Islander       6
Name: count, dtype: int64

## Part 4: END
- The resultant DataFrame contains an enrollment file for new students in Fall term 2021 only enriched with:
    - campus names
    - credential objectives
    - race/ethnicity

## Part 5: Level 3 - Calculate race percent by campus

In [40]:
# race by campus
sql_df = sqldf(
    """
    SELECT 
        campus_name,
        race_description,
        COUNT(calstateEduPersonUID) as total_records
        FROM df_2021_new_race
        GROUP BY 
            campus_name,
            race_description
        ORDER BY campus_name
            """
)

In [41]:
sql_df    

,campus_name,race_description,total_records
0,Bakersfield,Asian,2
1,Bakersfield,Black or African American,2
2,Bakersfield,Hispanic/Latino,106
3,Bakersfield,Two or More Races,4
4,Bakersfield,Unknown,10
...,...,...,...
136,Stanislaus,Black or African American,3
137,Stanislaus,Hispanic/Latino,119
138,Stanislaus,Two or More Races,5
139,Stanislaus,Unknown,11


In [42]:
# total race by campus
sql_partition_df = sqldf(
    """
    SELECT
        campus_name,
        race_description,
        total_records,
        SUM(total_records) OVER (PARTITION BY campus_name) AS total_campus_records
        FROM sql_df
"""
)

In [43]:
sql_partition_df

,campus_name,race_description,total_records,total_campus_records
0,Bakersfield,Asian,2,149
1,Bakersfield,Black or African American,2,149
2,Bakersfield,Hispanic/Latino,106,149
3,Bakersfield,Two or More Races,4,149
4,Bakersfield,Unknown,10,149
...,...,...,...,...
136,Stanislaus,Black or African American,3,211
137,Stanislaus,Hispanic/Latino,119,211
138,Stanislaus,Two or More Races,5,211
139,Stanislaus,Unknown,11,211


In [44]:
# total race percent by campus
sql_percent_df = sqldf(
    """
    SELECT
        campus_name,
        race_description,
        total_records,
        total_campus_records,
        ROUND((CAST(total_records AS REAL)/CAST(total_campus_records AS REAL))*100, 1) AS total_percent_campus_records
        FROM sql_partition_df
"""
)

In [45]:
sql_percent_df

,campus_name,race_description,total_records,total_campus_records,total_percent_campus_records
0,Bakersfield,Asian,2,149,1.3
1,Bakersfield,Black or African American,2,149,1.3
2,Bakersfield,Hispanic/Latino,106,149,71.1
3,Bakersfield,Two or More Races,4,149,2.7
4,Bakersfield,Unknown,10,149,6.7
...,...,...,...,...,...
136,Stanislaus,Black or African American,3,211,1.4
137,Stanislaus,Hispanic/Latino,119,211,56.4
138,Stanislaus,Two or More Races,5,211,2.4
139,Stanislaus,Unknown,11,211,5.2


In [46]:
sql_percent_df.to_csv("..\data\output\sql_percent_df_AY_2021.csv")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\2455078727.py:1: SyntaxWarning: invalid escape sequence '\d'
  sql_percent_df.to_csv("..\data\output\sql_percent_df_AY_2021.csv")


# Calculate race percent for system

In [47]:
# race by campus
sql_all = sqldf(
    """
    SELECT 
        'systemwide' as campus,
        race_description,
        COUNT(calstateEduPersonUID) as total_records
        FROM df_2021_new_race
        GROUP BY 
            race_description
            """
)

In [48]:
sql_all

,campus,race_description,total_records
0,systemwide,American Indian or Alaska Native,9
1,systemwide,Asian,430
2,systemwide,Black or African American,109
3,systemwide,Hispanic/Latino,2350
4,systemwide,Native Hawaiian or Other Pacific Islander,6
5,systemwide,Two or More Races,165
6,systemwide,Unknown,203
7,systemwide,White,1629


In [49]:
sql_system = sqldf(
    """
    SELECT 
        campus,
        race_description,
        total_records,
        SUM(total_records) OVER(PARTITION BY campus),
        (ROUND(100.0*total_records/(SUM(total_records) OVER(PARTITION BY campus)),1)) AS percent_of_total
        FROM sql_all
            """
)

In [50]:
sql_system

,campus,race_description,total_records,SUM(total_records) OVER(PARTITION BY campus),percent_of_total
0,systemwide,American Indian or Alaska Native,9,4901,0.2
1,systemwide,Asian,430,4901,8.8
2,systemwide,Black or African American,109,4901,2.2
3,systemwide,Hispanic/Latino,2350,4901,47.9
4,systemwide,Native Hawaiian or Other Pacific Islander,6,4901,0.1
5,systemwide,Two or More Races,165,4901,3.4
6,systemwide,Unknown,203,4901,4.1
7,systemwide,White,1629,4901,33.2


In [51]:
# append system_df onto campus_df

sql_combined_df_with_cst = sqldf(
    """
    SELECT *
    FROM sql_system
    UNION
    SELECT *
    FROM sql_percent_df
            """
)

In [52]:
sql_combined_df_with_cst.to_csv("..\data\output\RAW_level_3_first_time_epp_enrollment_fall_2016.csv")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\3294150127.py:1: SyntaxWarning: invalid escape sequence '\d'
  sql_combined_df_with_cst.to_csv("..\data\output\RAW_level_3_first_time_epp_enrollment_fall_2016.csv")


In [53]:
# insert rows for missing race/ethnicity

# Calculate for system without CalStateTEACH

In [54]:
# race by campus
sql_all_no_cst = sqldf(
    """
    SELECT 
        'systemwide' AS campus,
        race_description,
        COUNT(calstateEduPersonUID) as total_records
        FROM df_2021_new_race
        WHERE campus_name != 'CalStateTEACH'
        GROUP BY 
            race_description
            """
)

In [55]:
sql_all_no_cst

,campus,race_description,total_records
0,systemwide,American Indian or Alaska Native,7
1,systemwide,Asian,406
2,systemwide,Black or African American,99
3,systemwide,Hispanic/Latino,2225
4,systemwide,Native Hawaiian or Other Pacific Islander,5
5,systemwide,Two or More Races,152
6,systemwide,Unknown,187
7,systemwide,White,1482


In [56]:
sql_system_no_cst = sqldf(
    """
    SELECT 
        race_description,
        total_records,
        SUM(total_records) OVER(PARTITION BY campus),
        (ROUND(100.0*total_records/(SUM(total_records) OVER(PARTITION BY campus)),1)) AS percent_of_total
        FROM sql_all_no_cst
            """
)

In [57]:
sql_system_no_cst

,race_description,total_records,SUM(total_records) OVER(PARTITION BY campus),percent_of_total
0,American Indian or Alaska Native,7,4563,0.2
1,Asian,406,4563,8.9
2,Black or African American,99,4563,2.2
3,Hispanic/Latino,2225,4563,48.8
4,Native Hawaiian or Other Pacific Islander,5,4563,0.1
5,Two or More Races,152,4563,3.3
6,Unknown,187,4563,4.1
7,White,1482,4563,32.5


## Part 5: End

## Part 6: Level 4 - Connect Enrollment File to Two Years of Completer Files
- completers in AY 2021-2022 or AY 2022-2023
- this is used to determine who completed within 2 years

In [58]:
# read in AY 2021-2022
df_21_22 = pd.read_csv("..\data\completer lists\EDQ_COMB_COMP_2122_160623.csv")
# read in AY 2022-2023
print(len(df_21_22))

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\1210033779.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_21_22 = pd.read_csv("..\data\completer lists\EDQ_COMB_COMP_2122_160623.csv")


5344


In [59]:
df_21_22.compCohort.value_counts()

compCohort
2021-2022    5344
Name: count, dtype: int64

In [61]:
df_all = sqldf(
    """
    SELECT *
    FROM df_2021_new_race AS a
    LEFT JOIN df_21_22 AS b
    USING(calstateEduPersonUID)
    WHERE a.erss_campus != 96 
    """
)

In [62]:
# df_all.to_csv("..\data\output\completers_2021_2022.csv")

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\486548406.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_all.to_csv("..\data\output\completers_2021_2022.csv")


In [83]:
# Include all campuses except:
## CalStateTEACH
## Humboldt
## Channel Islands
### This means we're looking at 20 campuses for EPP levels.
### Blank SSN's were assigned a value of DBBA60960846556. This is because empty values are padded with zeroes by Ching.
#### An empty record will be padded with '000000000' and then swapped with 'DBBA60960846556'

In [4]:
os.chdir("Z:\\02 EdQ DataView and TPDM\\New Pipeline Data\\scripts")

In [5]:
# read in AY 2021-2022
df_21_22 = pd.read_csv("..\data\completer lists\EDQ_COMB_COMP_2122_160623.csv")
# read in AY 2022-2023
df_22_23 = pd.read_csv("..\data\completer lists\EDQ_COMB_COMP_2223_050324.csv")
print(len(df_21_22))
print(len(df_22_23))

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:4: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_43012\447725702.py:2: SyntaxWarning: invalid escape sequence '\d'
  df_21_22 = pd.read_csv("..\data\completer lists\EDQ_COMB_COMP_2122_160623.csv")
C:\Users\jluong\AppData\Local\Temp\ipykernel_43012\447725702.py:4: SyntaxWarning: invalid escape sequence '\d'
  df_22_23 = pd.read_csv("..\data\completer lists\EDQ_COMB_COMP_2223_050324.csv")


5344
5083


In [6]:
# drop columns so that both DataFrames have the same columns

df_21_22 = df_21_22.drop(columns =['Study Code'])

In [7]:
# drop columns so that both DataFrames have the same columns

df_22_23 = df_22_23.drop(columns = [
    'Personal Non-CSU Email',
    'Other Email',
    'Phone Number',
    'Noyce Grant Recipients',
    'Noyce Code',
    'campus_name',
    'campus_code'
])

In [8]:
# sort columns A to Z so they are in the same order for the UNION
df_21_22 = df_21_22.sort_index(axis=1)

In [9]:
# sort columns A to Z so they are in the same order for the UNION
df_22_23 = df_22_23.sort_index(axis=1)

In [10]:
# Find columns that are in df1 but not in df2
columns_not_in_df2 = set(df_21_22.columns) - set(df_22_23.columns)
print(columns_not_in_df2)

set()


In [11]:
# Find columns that are in df1 but not in df2
columns_not_in_df1 = set(df_22_23.columns) - set(df_21_22.columns)
print(columns_not_in_df1)

set()


In [12]:
assert df_21_22.columns.equals(df_22_23.columns), "Column names of the two DataFrames are not the same."

In [13]:
# append df_22_23 onto df_21_22

sql_completers_21_23 = sqldf(
    """
    SELECT *
    FROM df_21_22
    UNION
    SELECT *
    FROM df_22_23
            """
)

In [14]:
len(sql_completers_21_23)

10423

In [15]:
sql_completers_21_23.head()

,CBEST,Campus ID Number,Campus Letter Code,Cohort Number,Content Code,Credential Code,Credential Program,DOB,Evaluation Year,First Name,Last Name,Middle,Program Name,Program Type,Reporting Group,System ID Number,System Letter Code,calstateEduPersonUID,compCohort
0,None,1,BA,23,ART,S,P,1991-01-01,2023,Heather,Gospich,B,Traditional Single Subject Credential Program,T,3,1,CSU,DDAC06992224214,2021-2022
1,None,1,BA,23,ART,S,P,1991-02-28,2023,Berenice,Enriquez,R,Traditional Single Subject Credential Program,T,3,1,CSU,DFFA80850121914,2021-2022
2,None,1,BA,23,ART,S,P,1997-02-10,2023,Kimberly,Aguayo,R,Traditional Single Subject Credential Program,T,3,1,CSU,BCDC09256947941,2021-2022
3,None,1,BA,23,ART,S,X,1983-06-17,2023,Antonia,Lopez,A,Teacher Residency for Rural Education (TRRE),R,3,1,CSU,DCAE02203478492,2021-2022
4,None,1,BA,23,BUSI,E,P,1996-09-08,2023,Luis,Martinez,G,Ed. Spec. (Mild/Moderate) - Student Teaching,T,3,1,CSU,DFCB91500849441,2021-2022


In [17]:
sql_completers_21_23.to_csv("..\data\output\\for_schusterman_completers_21_to_23.csv", index = False)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_43012\2626708461.py:1: SyntaxWarning: invalid escape sequence '\d'
  sql_completers_21_23.to_csv("..\data\output\\for_schusterman_completers_21_to_23.csv", index = False)


In [96]:
# 96 is CalStateTEACH
# there are 20 completers who may not match with df_2021_new_race because the SSN was blank
# after joining these tables, 17 records were duplicated because they were found in 2 different completer lists
# 1 student, DEDF71281814529, showed an enrollment at 2 campuses, but only completed at Long Beach that year.
# business rule: Only keep the first record (earliest record) of completion
## What does this mean? This means that some campuses may have included a completer in both the 2021-2022 AND 2022-2023 lists.
# If someone completed multiple programs, we look at their first completed program for the 2-year completion rate.
df_all = sqldf(
    """
    SELECT *
    FROM df_2021_new_race AS a
    LEFT JOIN sql_completers_21_23 AS b
    USING(calstateEduPersonUID)
    WHERE a.erss_campus != 96 
    """
)

In [97]:
df_all.to_csv("..\data\output\RAW_level_4_epp_completers.csv", index = False)

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\jluong\AppData\Local\Temp\ipykernel_18212\3161568363.py:1: SyntaxWarning: invalid escape sequence '\d'
  df_all.to_csv("..\data\output\RAW_level_4_epp_completers.csv", index = False)


: 

In [ ]:
# feedback:
# first-time freshmen fall 2016
# 2-Year Completers or Completers within 2 years
# Share data tables
# Share funnel with numbers
# Remember that in some charts n<10 is NOT shown, which could affect total counts.

In [73]:
# calculate 1-year completion rate


In [74]:
# calculate 2-year completion rate